## 1 Itrodução

Nosso objetivo com esse projeto será criar um modelo para prever os sobreviventes do famoso desastre com o navio Titánic.  
Os dados estão na pasta [datasets](./datasets/), divididos em dois arquivos ``` .csv ``` , [train](./datasets/train.csv) e [test](./datasets/test.csv).  

## 2 Imports

Vamos começar importando as bibliotecas necessárias do nosso ambiente Conda. Esse é um problema de classificação binária (sobreviventes e não sobreviventes), então vamos utilizar uma LogisticRegression por ser um modelo simples de usar. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

## 3 Explorando o dataset

Vamos criar um dataframe com os nossos dados e visulizá-los.

In [2]:
df_train = pd.read_csv('./datasets/train.csv')
df_test = pd.read_csv('./datasets/test.csv')

In [4]:
display(df_train.head(10))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [5]:
display(df_test.head(10))

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


Vamos explorar os conjunto de treino ``` df_train ``` e teste ``` df_test ``` em busca de valores faltantes e, caso necessário, descartar algumas colunas.

In [10]:
df_train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [20]:
df_test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

## 4 Tratamento dos dados

A coluna 'Cabin' será descartada, pois está faltando grande parte dos seus dados. 'Age' será preenchida utilizando a mediana, enquanto que em 'Embarked' utilizaremos a moda.

In [11]:
df_train = df_train.drop(columns=['Cabin'])

df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())

df_train['Embarked'] = df_train['Embarked'].fillna(df_train['Embarked'].mode()[0])

In [12]:
df_train.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     891 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 76.7+ KB


As colunas 'Sex' e 'Embarked' serão interessantes para treinar o modelo, porém estão em formato de texto. Vamos convertê-las para inteiro.

In [14]:
df_train['Sex'] = df_train['Sex'].map({'male': 0, 'female': 1})

df_train['Embarked'] = df_train['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

In [15]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Embarked     891 non-null    int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 76.7+ KB


Agora vamos selecionar apenas as colunas que serão interessantes para treinar o modelo.

In [16]:
x_train = df_train[['Age', 'Fare', 'Sex', 'Pclass', 'Embarked']]
y_train = df_train['Survived']

In [17]:
display(x_train.head(10))

,Age,Fare,Sex,Pclass,Embarked
0,22.0,7.2500,0,3,0
1,38.0,71.2833,1,1,1
2,26.0,7.9250,1,3,0
3,35.0,53.1000,1,1,0
4,35.0,8.0500,0,3,0
5,28.0,8.4583,0,3,2
6,54.0,51.8625,0,1,0
7,2.0,21.0750,0,3,0
8,27.0,11.1333,1,3,0
9,14.0,30.0708,1,2,1


In [19]:
display(y_train.head(10))

0    0
1    1
2    1
3    1
4    0
5    0
6    0
7    0
8    1
9    1
Name: Survived, dtype: int64

Esse processo também será repetido no conjunto de teste.

In [21]:
df_test = df_test.drop(columns=['Cabin'])

df_test['Age'] = df_test['Age'].fillna(df_test['Age'].median())

df_test['Fare'] = df_test['Fare'].fillna(df_test['Fare'].mean())

df_test['Sex'] = df_test['Sex'].map({'male': 0, 'female': 1})

df_test['Embarked'] = df_test['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

x_test = df_test[['Age', 'Fare', 'Sex', 'Pclass', 'Embarked']]

display(x_test.head(10))

,Age,Fare,Sex,Pclass,Embarked
0,34.5,7.8292,0,3,2
1,47.0,7.0000,1,3,0
2,62.0,9.6875,0,2,2
3,27.0,8.6625,0,3,0
4,22.0,12.2875,1,3,0
5,14.0,9.2250,0,3,0
6,30.0,7.6292,1,3,2
7,26.0,29.0000,0,2,0
8,18.0,7.2292,1,3,1
9,21.0,24.1500,0,3,0


## 5 Treinar o modelo

Agora vamos treinar nosso modelo.

In [23]:
model = LogisticRegression()
model.fit(x_train, y_train)

LogisticRegression()

Essa é a parte mais simples. Com duas linhas de código, temos um modelo treinado.

## 6 Realizar previsões

Vamos fazer nossas previsões utilizando o modelo. 

In [24]:
predictions = model.predict(x_test)

In [26]:
df_test['Survived'] = predictions

In [27]:
display(df_test.head(10))

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Survived
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,2,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,0,0
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,2,0
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,0,1
5,897,3,"Svensson, Mr. Johan Cervin",0,14.0,0,0,7538,9.2250,0,0
6,898,3,"Connolly, Miss. Kate",1,30.0,0,0,330972,7.6292,2,1
7,899,2,"Caldwell, Mr. Albert Francis",0,26.0,1,1,248738,29.0000,0,0
8,900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",1,18.0,0,0,2657,7.2292,1,1
9,901,3,"Davies, Mr. John Samuel",0,21.0,2,0,A/4 48871,24.1500,0,0


## 7 Pseudo avaliação

Vamos agrupar por 'Pclass', para ver qual classe tem maior probablidade de ter sobrevieventes segundo o modelo.

In [30]:
survivors_by_class = df_test.groupby('Pclass')['Survived'].sum()

total_by_class = df_test.groupby('Pclass')['PassengerId'].count()

survival_rate_by_class = (survivors_by_class / total_by_class) * 100

display(survival_rate_by_class)

Pclass
1    59.813084
2    32.258065
3    29.816514
dtype: float64